In [31]:
import requests
import random
import time
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Optional
import json

# Configuration
OMS_BASE_URL = "http://localhost:8090"
COMMAND_API = f"{OMS_BASE_URL}/api/command/execute"

In [32]:
# NASDAQ Top Stocks
NASDAQ_STOCKS = [
    {"symbol": "AAPL", "securityId": "AAPL.OQ", "desc": "Apple Inc", "price_range": (170, 195)},
    {"symbol": "MSFT", "securityId": "MSFT.OQ", "desc": "Microsoft Corporation", "price_range": (370, 420)},
    {"symbol": "GOOGL", "securityId": "GOOGL.OQ", "desc": "Alphabet Inc Class A", "price_range": (140, 165)},
    {"symbol": "AMZN", "securityId": "AMZN.OQ", "desc": "Amazon.com Inc", "price_range": (175, 200)},
    {"symbol": "NVDA", "securityId": "NVDA.OQ", "desc": "NVIDIA Corporation", "price_range": (450, 550)},
    {"symbol": "META", "securityId": "META.OQ", "desc": "Meta Platforms Inc", "price_range": (480, 550)},
    {"symbol": "TSLA", "securityId": "TSLA.OQ", "desc": "Tesla Inc", "price_range": (240, 280)},
    {"symbol": "INTC", "securityId": "INTC.OQ", "desc": "Intel Corporation", "price_range": (35, 50)},
    {"symbol": "AMD", "securityId": "AMD.OQ", "desc": "Advanced Micro Devices", "price_range": (140, 170)},
    {"symbol": "NFLX", "securityId": "NFLX.OQ", "desc": "Netflix Inc", "price_range": (450, 520)},
    {"symbol": "PYPL", "securityId": "PYPL.OQ", "desc": "PayPal Holdings Inc", "price_range": (60, 85)},
    {"symbol": "ADBE", "securityId": "ADBE.OQ", "desc": "Adobe Inc", "price_range": (480, 550)},
    {"symbol": "CSCO", "securityId": "CSCO.OQ", "desc": "Cisco Systems Inc", "price_range": (48, 58)},
    {"symbol": "QCOM", "securityId": "QCOM.OQ", "desc": "Qualcomm Inc", "price_range": (160, 190)},
    {"symbol": "COST", "securityId": "COST.OQ", "desc": "Costco Wholesale Corp", "price_range": (720, 800)},
]

# NYSE Top Stocks  
NYSE_STOCKS = [
    {"symbol": "JPM", "securityId": "JPM.N", "desc": "JPMorgan Chase & Co", "price_range": (180, 210), "exchange": "XNYS"},
    {"symbol": "V", "securityId": "V.N", "desc": "Visa Inc", "price_range": (270, 300), "exchange": "XNYS"},
    {"symbol": "JNJ", "securityId": "JNJ.N", "desc": "Johnson & Johnson", "price_range": (150, 165), "exchange": "XNYS"},
    {"symbol": "WMT", "securityId": "WMT.N", "desc": "Walmart Inc", "price_range": (160, 180), "exchange": "XNYS"},
    {"symbol": "PG", "securityId": "PG.N", "desc": "Procter & Gamble Co", "price_range": (155, 175), "exchange": "XNYS"},
    {"symbol": "MA", "securityId": "MA.N", "desc": "Mastercard Inc", "price_range": (450, 500), "exchange": "XNYS"},
    {"symbol": "HD", "securityId": "HD.N", "desc": "Home Depot Inc", "price_range": (360, 400), "exchange": "XNYS"},
    {"symbol": "BAC", "securityId": "BAC.N", "desc": "Bank of America Corp", "price_range": (35, 45), "exchange": "XNYS"},
    {"symbol": "DIS", "securityId": "DIS.N", "desc": "Walt Disney Co", "price_range": (95, 115), "exchange": "XNYS"},
    {"symbol": "KO", "securityId": "KO.N", "desc": "Coca-Cola Co", "price_range": (58, 68), "exchange": "XNYS"},
    {"symbol": "PFE", "securityId": "PFE.N", "desc": "Pfizer Inc", "price_range": (26, 35), "exchange": "XNYS"},
    {"symbol": "XOM", "securityId": "XOM.N", "desc": "Exxon Mobil Corp", "price_range": (105, 125), "exchange": "XNYS"},
    {"symbol": "CVX", "securityId": "CVX.N", "desc": "Chevron Corp", "price_range": (145, 165), "exchange": "XNYS"},
    {"symbol": "NKE", "securityId": "NKE.N", "desc": "Nike Inc", "price_range": (75, 95), "exchange": "XNYS"},
    {"symbol": "MCD", "securityId": "MCD.N", "desc": "McDonald's Corp", "price_range": (280, 310), "exchange": "XNYS"},
]

# Combine all stocks
ALL_STOCKS = NASDAQ_STOCKS + NYSE_STOCKS

print(f"Loaded {len(NASDAQ_STOCKS)} NASDAQ stocks and {len(NYSE_STOCKS)} NYSE stocks")

Loaded 15 NASDAQ stocks and 15 NYSE stocks


In [33]:
# Order parameters
ACCOUNTS = [f"account-{i:02d}" for i in range(1, 11)]  # 10 accounts
SIDES = ["BUY", "SELL"]
ORD_TYPES = ["LIMIT", "MARKET"]  # Removed STOP, STOP_LIMIT (require stopPx)
TIME_IN_FORCE = ["DAY", "GOOD_TILL_CANCEL", "IMMEDIATE_OR_CANCEL", "FILL_OR_KILL"]  # Fixed enum values
EXEC_INST = ["NO_CROSS", "OK_TO_CROSS", "MID_PEG"]  # Fixed enum values
HANDL_INST = ["AUTO", "BROKER_INTERVENTION_OK", "MANUAL"]  # Fixed enum values

def generate_order_id(prefix: str = "ORD") -> str:
    """Generate unique order ID"""
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")[:-3]
    random_suffix = random.randint(1000, 9999)
    return f"{prefix}-{timestamp}-{random_suffix}"

def generate_random_order(stock: Optional[dict] = None) -> dict:
    """Generate a random order"""
    if stock is None:
        stock = random.choice(ALL_STOCKS)
    
    # Determine exchange based on stock
    is_nasdaq = stock.get("exchange") is None or stock["securityId"].endswith(".OQ")
    exchange = "XNAS" if is_nasdaq else stock.get("exchange", "XNYS")
    
    # Random price within stock's range
    price_min, price_max = stock["price_range"]
    price = round(random.uniform(price_min, price_max), 2)
    
    # Random quantity (round lots)
    qty = random.choice([100, 200, 500, 1000, 2000, 5000])
    
    # Random timestamps
    now = datetime.utcnow()
    sending_time = now - timedelta(seconds=random.randint(0, 60))
    transact_time = sending_time - timedelta(seconds=random.randint(0, 30))
    
    order = {
        "sessionId": f"session-{random.randint(1, 5):02d}",
        "clOrdId": generate_order_id(),
        "sendingTime": sending_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "account": random.choice(ACCOUNTS),
        "execInst": random.choice(EXEC_INST),
        "handlInst": random.choice(HANDL_INST),
        "securityIdSource": "RIC",
        "orderQty": qty,
        "priceType": "PER_UNIT",
        "ordType": random.choice(ORD_TYPES),
        "price": price,
        "securityId": stock["securityId"],
        "securityDesc": stock["desc"],
        "side": random.choice(SIDES),
        "symbol": stock["symbol"],
        "timeInForce": random.choice(TIME_IN_FORCE),
        "transactTime": transact_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "exDestination": exchange,
        "settlCurrency": "USD",
        "securityExchange": exchange,
        "text": f"Auto-generated test order for {stock['symbol']}"
    }
    
    return order

# Test: Generate a sample order
sample = generate_random_order()
print(json.dumps(sample, indent=2))

{
  "sessionId": "session-01",
  "clOrdId": "ORD-20251130191137992-1316",
  "sendingTime": "2025-11-30T18:11:26Z",
  "account": "account-06",
  "execInst": "OK_TO_CROSS",
  "handlInst": "MANUAL",
  "securityIdSource": "RIC",
  "orderQty": 1000,
  "priceType": "PER_UNIT",
  "ordType": "LIMIT",
  "price": 60.42,
  "securityId": "KO.N",
  "securityDesc": "Coca-Cola Co",
  "side": "BUY",
  "symbol": "KO",
  "timeInForce": "FILL_OR_KILL",
  "transactTime": "2025-11-30T18:11:11Z",
  "exDestination": "XNYS",
  "settlCurrency": "USD",
  "securityExchange": "XNYS",
  "text": "Auto-generated test order for KO"
}


In [34]:
def send_order(order: dict) -> dict:
    """Send order to OMS Command API"""
    payload = {
        "type": "OrderCreateCmd",
        "order": order
    }
    
    try:
        response = requests.post(
            COMMAND_API,
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=10
        )
        return {
            "success": response.status_code in [200, 201, 202],
            "status_code": response.status_code,
            "response": response.json() if response.content else None,
            "order": order
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "order": order
        }

# Test: Send a single order
result = send_order(generate_random_order())
print(f"Success: {result['success']}")
print(f"Status: {result.get('status_code', 'N/A')}")
if result.get('response'):
    print(f"Response: {json.dumps(result['response'], indent=2)}")

Success: True
Status: 200
Response: {
  "id": "ORD-78ccba4e-8c51-4a57-ad3a-6fee10ef6576",
  "status": "OK",
  "message": "Order created successfully: ORD-78ccba4e-8c51-4a57-ad3a-6fee10ef6576"
}


In [35]:
def generate_orders_batch(
    count: int = 10,
    delay_ms: int = 100,
    stock_filter: Optional[str] = None,
    side_filter: Optional[str] = None,
    verbose: bool = True
) -> list:
    """Generate and send a batch of orders"""
    results = []
    success_count = 0
    
    # Filter stocks if specified
    stocks = ALL_STOCKS
    if stock_filter:
        stocks = [s for s in ALL_STOCKS if stock_filter.upper() in s["symbol"]]
        if not stocks:
            print(f"No stocks match filter: {stock_filter}")
            return []
    
    print(f"\n{'='*60}")
    print(f" Generating {count} orders")
    if stock_filter:
        print(f" Stock filter: {stock_filter}")
    if side_filter:
        print(f" Side filter: {side_filter}")
    print(f"{'='*60}\n")
    
    start_time = time.time()
    
    for i in range(count):
        order = generate_random_order(random.choice(stocks))
        
        # Apply side filter if specified
        if side_filter:
            order["side"] = side_filter.upper()
        
        result = send_order(order)
        results.append(result)
        
        if result["success"]:
            success_count += 1
            if verbose:
                print(f"[{i+1:4d}/{count}] ✓ {order['side']:4s} {order['orderQty']:5d} {order['symbol']:5s} @ ${order['price']:.2f}")
        else:
            if verbose:
                print(f"[{i+1:4d}/{count}] ✗ {order['symbol']} - {result.get('error', 'Failed')}")
        
        if delay_ms > 0 and i < count - 1:
            time.sleep(delay_ms / 1000)
    
    elapsed = time.time() - start_time
    
    print(f"\n{'='*60}")
    print(f" Results: {success_count}/{count} orders created successfully")
    print(f" Time: {elapsed:.2f}s ({count/elapsed:.1f} orders/sec)")
    print(f"{'='*60}\n")
    
    return results

## Generate Orders

Run the cells below to generate orders. Adjust parameters as needed.

In [36]:
# Generate 10 random orders (quick test)
results = generate_orders_batch(count=10, delay_ms=50)


 Generating 10 orders

[   1/10] ✓ SELL   500 PFE   @ $27.32
[   2/10] ✓ SELL  2000 PYPL  @ $60.74
[   3/10] ✓ SELL  2000 AMZN  @ $195.27
[   4/10] ✓ SELL   100 ADBE  @ $495.52
[   5/10] ✓ BUY    100 V     @ $293.09
[   6/10] ✓ SELL   500 GOOGL @ $147.39
[   7/10] ✓ BUY    500 WMT   @ $177.40
[   8/10] ✓ BUY   5000 AAPL  @ $181.19
[   9/10] ✓ SELL   500 COST  @ $792.15
[  10/10] ✓ BUY   5000 AAPL  @ $171.29

 Results: 10/10 orders created successfully
 Time: 0.73s (13.7 orders/sec)



In [37]:
# Generate 50 orders for a specific stock
results = generate_orders_batch(count=50, delay_ms=20, stock_filter="INTC")


 Generating 50 orders
 Stock filter: INTC

[   1/50] ✓ BUY   2000 INTC  @ $46.91
[   2/50] ✓ SELL  2000 INTC  @ $45.62
[   3/50] ✓ SELL  1000 INTC  @ $36.24
[   4/50] ✓ SELL   100 INTC  @ $35.81
[   5/50] ✓ SELL  2000 INTC  @ $35.92
[   6/50] ✓ BUY    100 INTC  @ $43.06
[   7/50] ✓ SELL   100 INTC  @ $44.12
[   8/50] ✓ SELL  5000 INTC  @ $39.78
[   9/50] ✓ BUY   2000 INTC  @ $43.69
[  10/50] ✓ SELL  2000 INTC  @ $47.06
[  11/50] ✓ BUY    200 INTC  @ $36.18
[  12/50] ✓ BUY   1000 INTC  @ $48.44
[  13/50] ✓ BUY   2000 INTC  @ $44.82
[  14/50] ✓ SELL  1000 INTC  @ $41.73
[  15/50] ✓ BUY    100 INTC  @ $39.68
[  16/50] ✓ BUY    100 INTC  @ $45.45
[  17/50] ✓ SELL  1000 INTC  @ $46.05
[  18/50] ✓ SELL  1000 INTC  @ $45.44
[  19/50] ✓ SELL  1000 INTC  @ $38.96
[  20/50] ✓ SELL   200 INTC  @ $38.52
[  21/50] ✓ BUY    100 INTC  @ $48.05
[  22/50] ✓ SELL   200 INTC  @ $48.99
[  23/50] ✓ BUY   5000 INTC  @ $41.95
[  24/50] ✓ SELL  5000 INTC  @ $38.56
[  25/50] ✓ BUY    200 INTC  @ $47.80
[  26/

In [38]:
# Generate 100 BUY orders across all stocks
results = generate_orders_batch(count=100, delay_ms=10, side_filter="BUY")


 Generating 100 orders
 Side filter: BUY

[   1/100] ✓ BUY   1000 TSLA  @ $244.37
[   2/100] ✓ BUY    200 PYPL  @ $84.63
[   3/100] ✓ BUY   5000 DIS   @ $95.84
[   4/100] ✓ BUY   1000 V     @ $291.45
[   5/100] ✓ BUY   5000 MSFT  @ $380.85
[   6/100] ✓ BUY    100 NVDA  @ $531.84
[   7/100] ✓ BUY    200 COST  @ $756.23
[   8/100] ✓ BUY    500 CSCO  @ $50.95
[   9/100] ✓ BUY   2000 AMD   @ $141.24
[  10/100] ✓ BUY   5000 DIS   @ $104.14
[  11/100] ✓ BUY    100 AMD   @ $143.19
[  12/100] ✓ BUY   5000 BAC   @ $36.24
[  13/100] ✓ BUY    100 AMZN  @ $182.96
[  14/100] ✓ BUY    200 INTC  @ $47.70
[  15/100] ✓ BUY   1000 NKE   @ $83.34
[  16/100] ✓ BUY   1000 CSCO  @ $57.07
[  17/100] ✓ BUY    500 DIS   @ $108.77
[  18/100] ✓ BUY   5000 NKE   @ $85.07
[  19/100] ✓ BUY    200 DIS   @ $102.92
[  20/100] ✓ BUY   5000 COST  @ $752.80
[  21/100] ✓ BUY    100 WMT   @ $172.57
[  22/100] ✓ BUY    100 WMT   @ $174.19
[  23/100] ✓ BUY   1000 PYPL  @ $67.55
[  24/100] ✓ BUY   5000 MCD   @ $308.68
[  25/

In [39]:
# Heavy load: 500 orders with minimal delay
results = generate_orders_batch(count=500, delay_ms=5, verbose=False)


 Generating 500 orders


 Results: 500/500 orders created successfully
 Time: 16.84s (29.7 orders/sec)



## Stock Distribution Analysis

Analyze the distribution of generated orders by symbol and exchange.

In [40]:
from collections import Counter

if results:
    symbols = [r["order"]["symbol"] for r in results if r["success"]]
    sides = [r["order"]["side"] for r in results if r["success"]]
    exchanges = [r["order"]["securityExchange"] for r in results if r["success"]]
    
    print("\nTop 10 Symbols:")
    for symbol, count in Counter(symbols).most_common(10):
        print(f"  {symbol:6s}: {count:4d} orders")
    
    print("\nSide Distribution:")
    for side, count in Counter(sides).items():
        print(f"  {side:6s}: {count:4d} orders")
    
    print("\nExchange Distribution:")
    for exchange, count in Counter(exchanges).items():
        print(f"  {exchange:6s}: {count:4d} orders")


Top 10 Symbols:
  CSCO  :   23 orders
  NFLX  :   22 orders
  NKE   :   22 orders
  PFE   :   22 orders
  JNJ   :   21 orders
  DIS   :   21 orders
  CVX   :   19 orders
  MA    :   19 orders
  TSLA  :   19 orders
  AMD   :   19 orders

Side Distribution:
  SELL  :  252 orders
  BUY   :  248 orders

Exchange Distribution:
  XNAS  :  244 orders
  XNYS  :  256 orders
